In [3]:
import freenect
import cv2
import numpy as np
import pygame
import matplotlib.pyplot as plt
from scipy import stats

In [4]:
def get_video():
    array,_ = freenect.sync_get_video()
    array = cv2.cvtColor(array,cv2.COLOR_RGB2BGR)
    return array
 
#function to get depth image from kinect
def get_depth():
    array,_ = freenect.sync_get_depth()
#     np.clip(array, 0, 2**10-1, array)
    array = array.astype(np.uint8)
    return array

def imshow(img, title = ''):
    # hide the x and y axis for images
    plt.axis('off')
    # RGB images are actually BGR in OpenCV, so convert before displaying
    if len(img.shape) == 3: 
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # otherwise, assume it's grayscale and just display it
    else:
        plt.imshow(img,cmap='gray')
    # add a title if specified
    plt.title(title)
    plt.show()

In [14]:
def deg2rad(angle_deg):
    """Convert degrees to radians
        This method converts an angle in radians e[0,2*np.pi) into degrees
        e[0,360)
    """
    return angle_deg/180.0*np.pi

def angle_rad(v1, v2):
    """Angle in radians between two vectors
        This method returns the angle (in radians) between two array-like
        vectors using the cross-product method, which is more accurate for
        small angles than the dot-product-acos method.
    """
    return np.arctan2(np.linalg.norm(np.cross(v1, v2)), np.dot(v1, v2))

def detect_num_fingers(contours, defects, img_draw):
    """Detects the number of extended fingers
        This method determines the number of extended fingers based on a
        contour and convexity defects.
        It will annotate an RGB color image of the segmented arm region
        with all relevant defect points and the hull.
        :param contours: a list of contours
        :param defects: a list of convexity defects
        :param img_draw: an RGB color image to be annotated
        :returns: (num_fingers, img_draw) the estimated number of extended
                  fingers and an annotated RGB color image
    """

    # if there are no convexity defects, possibly no hull found or no
    # fingers extended
    thresh_deg = 80
    if defects is None:
        return [0, img_draw]

    # we assume the wrist will generate two convexity defects (one on each
    # side), so if there are no additional defect points, there are no
    # fingers extended
    if len(defects) <= 2:
        return [0, img_draw]

    # if there is a sufficient amount of convexity defects, we will find a
    # defect point between two fingers so to get the number of fingers,
    # start counting at 1
    num_fingers = 1

    for i in range(defects.shape[0]):
        # each defect point is a 4-tuple
        start_idx, end_idx, farthest_idx, _ = defects[i, 0]
        start = tuple(contours[start_idx][0])
        end = tuple(contours[end_idx][0])
        far = tuple(contours[farthest_idx][0])

        # draw the hull
        cv2.line(img_draw, start, end, [0, 255, 0], 2)

        # if angle is below a threshold, defect point belongs to two
        # extended fingers
        if angle_rad(np.subtract(start, far),
                     np.subtract(end, far)) < deg2rad(thresh_deg):
            # increment number of fingers
            num_fingers = num_fingers + 1

            # draw point as green
            cv2.circle(img_draw, far, 5, [0, 255, 0], -1)
        else:
            # draw point as red
            cv2.circle(img_draw, far, 5, [255, 0, 0], -1)

    # make sure we cap the number of fingers
    cv2.putText(img_draw,str(num_fingers),(10,500), cv2.FONT_HERSHEY_SIMPLEX, 4,(0,0,255),2,cv2.LINE_AA)
    return (min(5, num_fingers), img_draw)


def find_hull_defects(segment):
    """Find hull defects
        This method finds all defects in the hull of a segmented arm
        region.
        :param segment: a binary image (mask) of a segmented arm region,
                        where arm=255, else=0
        :returns: (max_contour, defects) the largest contour in the image
                  and all corresponding defects
    """
    _, contours, hierarchy = cv2.findContours(segment, cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)

    # find largest area contour
    max_contour = max(contours, key=cv2.contourArea)
    epsilon = 0.01*cv2.arcLength(max_contour, True)
    max_contour = cv2.approxPolyDP(max_contour, epsilon, True)

    # find convexity hull and defects
    hull = cv2.convexHull(max_contour, returnPoints=False)
    defects = cv2.convexityDefects(max_contour, hull)

    return (max_contour, defects)

def recognize(img_gray):
    segment = segment_arm(img_gray)
    (contours, defects) = find_hull_defects(segment)
    img_draw = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
    (num_fingers, img_draw) = detect_num_fingers(contours, defects, img_draw)
    return (num_fingers, img_draw)
    

def segment_arm(frame):
    """Segments arm region
        This method accepts a single-channel depth image of an arm and
        hand region and extracts the segmented arm region.
        It is assumed that the hand is placed in the center of the image.
        :param frame: single-channel depth image
        :returns: binary image (mask) of segmented arm region, where
                  arm=255, else=0
    """
    height, width = frame.shape
    abs_depth_dev = 15
    # find center (21x21 pixel) region of image frame
    center_half = 10  # half-width of 21 is 21/2-1
    center = frame[int(height/2)-center_half:int(height/2)+center_half,
                   int(width/2)-center_half:int(width/2)+center_half]

    # find median depth value of center region
    med_val = np.median(center)

    # try this instead:
    frame = np.where(abs(frame-med_val) <= abs_depth_dev,
                     128, 0).astype(np.uint8)

    # morphological
    kernel = np.ones((3, 3), np.uint8)
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel)

    # connected component
    small_kernel = 3
    frame[int(height/2)-small_kernel:int(height/2)+small_kernel,
         int(width/2)-small_kernel:int(width/2)+small_kernel] = 128

    mask = np.zeros((height+2, width+2), np.uint8)
    flood = frame.copy()
    cv2.floodFill(flood, mask, (int(width/2), int(height/2)), 255,
                  flags=4 | (255 << 8))

    ret, flooded = cv2.threshold(flood, 129, 255, cv2.THRESH_BINARY)

    return flooded

In [15]:
while 1:
    #get a frame from RGB camera
    frame = get_video()
    #get a frame from depth sensor
    depth = get_depth()
    height, width = depth.shape
#     cv2.imshow('RGB image',frame)
#     cv2.imshow('Depth image',depth)
    num_fingers, hand = recognize(depth)
    cv2.imshow("depth", hand)
#     num_fingers, img_draw = hand_gestures.recognize(depth)
#     cv2.rectangle(img_draw, (width/3, height/3), (width*2/3,height*2/3), [255, 102, 0], 2)

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()